In [ ]:
from google.colab import files
import os
import subprocess

uploaded = None
resize_to_720p = False

def upload_video():
  global uploaded
  global video_path
  uploaded = files.upload()
  for filename in uploaded.keys():
    print(f'Uploaded {filename}')
    if resize_to_720p:
        filename = resize_video(filename)
    video_path = filename
    return filename


def resize_video(filename):
    output_filename = f"resized_{filename}"
    cmd = f"ffmpeg -i {filename} -vf scale=-1:720 {output_filename}"
    subprocess.run(cmd, shell=True)
    print(f'Resized video saved as {output_filename}')
    return output_filename

import ipywidgets as widgets
from IPython.display import display

button = widgets.Button(description="Upload Video")
checkbox = widgets.Checkbox(value=False, description='Resize to 720p (better results)')
output = widgets.Output()

def on_button_clicked(b):
  with output:
    global video_path
    global resize_to_720p
    resize_to_720p = checkbox.value
    video_path = upload_video()

button.on_click(on_button_clicked)
display(checkbox, button, output)

In [1]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-xefwia61
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-xefwia61
ERROR: Operation cancelled by user


In [ ]:
import subprocess

# Ensure video_path variable exists
if 'video_path' in globals() and video_path is not None:
    ffmpeg_command = f"ffmpeg -i '{video_path}' -acodec pcm_s24le -ar 48000 -q:a 0 -map a -y 'output_audio.wav'"
    subprocess.run(ffmpeg_command, shell=True)
else:
    print("No video uploaded. Please upload a video first.")

import whisper

model = whisper.load_model("base")
result = model.transcribe("output_audio.wav")

whisper_text = result["text"]
whisper_language = result['language']

print("Audio text:", whisper_text)

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
target_language = "Hindi" # @param ["English","Spanish","French","German","Italian","Portuguese","Polish","Turkish","Russian","Dutch","Czech","Arabic","Chinese (Simplified)","Hindi"]

# Mapping between full names and ISO 639-1 codes
language_mapping = {
    'English': 'en',
    'Spanish': 'es',
    'French': 'fr',
    'German': 'de',
    'Italian': 'it',
    'Portuguese': 'pt',
    'Polish': 'pl',
    'Turkish': 'tr',
    'Russian': 'ru',
    'Dutch': 'nl',
    'Czech': 'cs',
    'Arabic': 'ar',
    'Chinese (Simplified)': 'zh-cn',
    'Hindi' : 'hi'
}

target_language_code = language_mapping[target_language]
from googletrans import Translator
translator = Translator()
translated_text = translator.translate(whisper_text, src=whisper_language, dest=target_language_code).text
print("Translated text:", translated_text)

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
from TTS.api import TTS
import torch
from IPython.display import Audio, display


# Use CPU if CUDA is not available
device = "cpu"  # Ensure the device is set to CPU

# Load the TTS model with gpu=False to make sure it doesn't attempt to use the GPU
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=False).to(device)


#generate audio
tts.tts_to_file(translated_text,
    speaker_wav='output_audio.wav',
    file_path="output_synth.wav",
    language=target_language_code
)
audio_widget = Audio(filename="output_synth.wav", autoplay=False)
display(audio_widget)

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_audioclips
from moviepy.audio.AudioClip import AudioArrayClip
import numpy as np

def merge_audio_with_video(video_path, audio_path, output_path, audio_delay=0):
    try:
        # Load the video file
        video = VideoFileClip(video_path)

        # Load the translated audio file
        audio = AudioFileClip(audio_path)

        # Apply audio delay
        if audio_delay > 0:
            # Create a silent audio clip
            sr = 44100  # Sample rate
            silence_duration = audio_delay  # In seconds
            silence = AudioArrayClip(
                np.zeros((int(sr * silence_duration), audio.nchannels)),
                fps=sr
            )

            # Combine silence and audio
            audio = concatenate_audioclips([silence, audio])
        elif audio_delay < 0:
            # Trim the audio to remove delay (ensure audio length > abs(delay))
            start_time = abs(audio_delay)
            if start_time < audio.duration:
                audio = audio.subclip(start_time)
            else:
                raise ValueError("Audio delay exceeds the audio's length.")

        # Set the audio of the video to the adjusted audio
        video = video.set_audio(audio)

        # Write the final video to a file
        video.write_videofile(output_path, codec="libx264", audio_codec="aac")

        print(f"Video with translated audio saved at {output_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# File paths
video_file = "/content/trumph.mp4"  # Replace with your video file path
audio_file = "/content/output_synth.wav"  # Replace with your translated audio file path
output_file = "final_video123.mp4"

# Set the audio delay (positive to add delay, negative to trim)
audio_delay = 1  # Example: 5 seconds delay
merge_audio_with_video(video_file, audio_file, output_file, audio_delay)